# LAB 5 - Quantum Key Distribution

| Name | Surname | ID Number |
|------|---------|---------------------|
| Lorenzo | Calandra Buonaura | 2107761 |

**Import zone**

In [1]:
import qkd_functions as qkd

**Dataset loading**

In [2]:
dataset = 2

transmitted_states = qkd.dataset_loading(f"./Data/set{dataset}/states.txt", t="state")
received_states = qkd.dataset_loading(f"./Data/set{dataset}/statesRCV.txt", t="state")
decoy = qkd.dataset_loading(f"./Data/set{dataset}/decoy.txt", t="decoy")

assert(len(received_states) == len(transmitted_states) == len(decoy))

**Probabilities**

In [3]:
a_values = ['D', 'H', 'V']
b_values = ['D', 'H', 'V', 'A']
decoy_values = ['H', 'L', 'N']

states_trans_sep = qkd.divide_states(transmitted_states, a_values)
states_rec_sep = qkd.divide_states(received_states, b_values)
states_decoy_sep = qkd.divide_states(decoy, decoy_values)

In [4]:
prob_enc_X = len(states_trans_sep['D']) / len(transmitted_states)
prob_enc_Z = (len(states_trans_sep['H']) + len(states_trans_sep['V'])) / len(transmitted_states)

print(f'Probability of encoding in base X = {prob_enc_X}')
print(f'Probability of encoding in base Z = {prob_enc_Z}')
print(f'Total probability = {prob_enc_Z + prob_enc_X}')

Probability of encoding in base X = 0.09460960144927537
Probability of encoding in base Z = 0.9053903985507247
Total probability = 1.0


In [5]:
prob_dec_X = (len(states_rec_sep['D']) + len(states_rec_sep['A'])) / len(received_states)
prob_dec_Z = (len(states_rec_sep['H']) + len(states_rec_sep['V'])) / len(received_states)

print(f'Probability of decoding in base X = {prob_dec_X}')
print(f'Probability of decoding in base Z = {prob_dec_Z}')
print(f'Total probability = {prob_dec_Z + prob_dec_X}')

Probability of decoding in base X = 0.5423509057971014
Probability of decoding in base Z = 0.45764909420289857
Total probability = 1.0


**Conditional probabilities**

In [6]:
cond_probs = qkd.total_cond_probs(transmitted_states, received_states, a_values, b_values)

In [7]:
cond_probs

{('D', 'D'): 0.3664582715009239,
 ('H', 'D'): 0.2765062374938965,
 ('V', 'D'): 0.19371942287623625,
 ('A', 'D'): 0.1633160681289433,
 ('D', 'H'): 0.1691587689001896,
 ('H', 'H'): 0.43101611254865296,
 ('V', 'H'): 0.041409403317757026,
 ('A', 'H'): 0.35841571523340043,
 ('D', 'V'): 0.3006719497032055,
 ('H', 'V'): 0.047275483870379206,
 ('V', 'V'): 0.38847058007887153,
 ('A', 'V'): 0.26358198634754376}

**QBER**

In [8]:
# QBER achieved in the state H, prob(a!=b|a=H)
a_val = a_values[1]
b_val = [b for b in b_values if b != a_val]

cond_probs, qber = qkd.getQBER(transmitted_states, received_states, a_val, b_val)
print(f'P(a!=b|a=H) = {qber}\n----------------------------')

# QBER achieved in the state V, prob(a!=b|a=V)
a_val = a_values[2]
b_val = [b for b in b_values if b != a_val]

cond_probs, qber = qkd.getQBER(transmitted_states, received_states, a_val, b_val)
print(f'P(a!=b|a=V) = {qber}\n----------------------------')

# QBER achieved in the state D, prob(a!=b|a=D)
a_val = a_values[0]
b_val = [b for b in b_values if b != a_val]

cond_probs, qber = qkd.getQBER(transmitted_states, received_states, a_val, b_val)
print(f'P(a!=b|a=D) = {qber}\n----------------------------')

# QBER achieved in the base X, prob(a!=b|A=X, B=X)
a_X = ['D']
b_X = ['A', 'D']
a_val = a_X[0]
b_val = [b for b in b_X if b != a_val]

cond_probs, qber = qkd.getQBER(transmitted_states, received_states, a_val, b_val)
print(f'P(a!=b|A=X, B=X) = {qber}\n----------------------------')

# QBER achieved in the base Z, prob(a!=b|A=Z, B=Z)
a_Z = ['H', 'V']
b_Z = ['H', 'V']

a_val = a_Z[0]
b_val = [b for b in b_Z if b != a_val]
cond_probs1, qber1 = qkd.getQBER(transmitted_states, received_states, a_val, b_val)

a_val = a_Z[1]
b_val = [b for b in b_Z if b != a_val]
cond_probs2, qber2 = qkd.getQBER(transmitted_states, received_states, a_val, b_val)
print(f'P(a!=b|A=Z, B=Z) = {qber1 + qber2}')

P(a!=b|a=H) = 0.5689838874513471
----------------------------
P(a!=b|a=V) = 0.6115294199211285
----------------------------
P(a!=b|a=D) = 0.6335417284990761
----------------------------
P(a!=b|A=X, B=X) = 0.1633160681289433
----------------------------
P(a!=b|A=Z, B=Z) = 0.08868488718813623


**Decoy state**

In [9]:
prob_decoy_high = len(states_decoy_sep['H']) / len(decoy)
prob_decoy_low = len(states_decoy_sep['L']) / len(decoy)

print(f'Probability of high intensity state = {prob_decoy_high}')
print(f'Probability of low intensity state = {prob_decoy_low}')
print(f'Total probability = {prob_decoy_high + prob_decoy_low}')

Probability of high intensity state = 0.8900028985507247
Probability of low intensity state = 0.10999710144927537
Total probability = 1.0


**Modified BB84**

Let's suppose that you have a perfect channel which does not introduce any errors, which means:

$P(a=b|A=B)=1$ and $P(b=D|A=Z)=P(b=A|A=Z)=P(b=H|A=X)=P(b=V|A=X)=0.5$.

An eavesdropper intercepts the photons along the channel fiber and he measures with the same probabilities of Bob in the base $E={X, Z}$.<br> 
After the measurement, he sends to Bob the state $e$ measured by him. <br>
What are the QBERs experienced by Alice and Bob in this intercept and resend attack? Is our system secure against this type of attack?

**Answer**

We have to consider each possibility, in particular starting from the two cases when $A=B$ and when $A\neq B$.

1. If $A=B$, then we have QBER when $a \neq b$. We have two cases also here, one when $A=B=E$ and one when $A=B \neq E$:
    - When $A=B=E$ then Eve is not introducing error, as it has the same probability as B, so we have that $P(a=b|A=B)=1$ <br>
      and consequently $P(a\neq b|A=B)=0$.
    - When $A=B \neq E$, then Eve is using a different basis, thus it will project the state on one of the possible states <br>
      of the other basis. Thus for B it's the same as using the wrong basis, so $P(b \neq a|A = B \neq E) = P(b \neq a|A\neq B) = 0.5$.


2. If $A \neq B$, then we have QBER when $a \neq b$. We have two cases also here, one when $A \neq B=E$ and one when $A=E \neq B$:
    - When $A \neq B=E$ then Eve is using a different basis, projecting the state on one of the possible states of the other basis. <br>
      The B uses the same basis of E, so will always obtain the same as E, thus we have that $P(a \neq b|A \neq E=B) = 0.5$.
    - When $A=E \neq B$, then Eve is not introducing an error and we'll simply have that $P(b \neq a|A = E \neq B) = 0.5$.

So in general we have that when $A \neq B$ Eve doesn't modify the probabilities, while when $A = B$ then Eve changes the probabilities. How much error do we have in total?
- For $A=B$ we have error only if $A=B \neq E$, which happens only half of the times. Thus we'll have that the total QBER in this case is:

  $QBER(A = B) = P(b \neq a|A = B \neq E) * P(A = B \neq E) = \frac{1}{2} * \frac{1}{2} = \frac{1}{4}$

- For $A \neq B$ we have error in both cases. Thus we'll have that the total QBER in this case is:

  $QBER(A \neq B) = P(b \neq a|A = E \neq B) * P(A = E \neq B) +  P(b \neq a|A \neq E = B) * P(A \neq E = B) = \frac{1}{2} * \frac{1}{2} + \frac{1}{2} * \frac{1}{2} = \frac{1}{2}$  

Finally the total QBER will be given by the average of these two errors:

$QBER_{TOT} = \frac{QBER(A = B) + QBER(A \neq B)}{2} = \frac{1}{2} * \left( \frac{1}{4} + \frac{1}{2} \right) = \frac{3}{8} = 0.375$

This total QBER is 37.5%, so it's much over the accepted threshold of 11% (as reported in the paper by Shor). Thus, we can conclude that the protocol <br> detects Eve's attack due to the abnormally high QBER: consequently, Alice and Bob would abort the key generation process, preventing Eve from compromising their security.